### Étude particulière du dataset "users-score-2023.csv" : 

In [13]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import gc

In [2]:
# Charger le dataset
user_score = pd.read_csv("./users-score-2023.csv")

### Restreindre le dataset

In [3]:
# Fixer les seuils
min_user_ratings = 50  # Par exemple, les utilisateurs doivent avoir au moins noté 50 animes
min_anime_ratings = 10_000  # Par exemple, les animes doivent avoir au moins 10 000 notes

# Compter le nombre de notes par utilisateur et par anime
user_rating_counts = user_score.groupby('user_id').size()
anime_rating_counts = user_score.groupby('anime_id').size()

# Filtrer les utilisateurs avec au moins 10 notes et les animes avec au moins 100,000 notes
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
filtered_animes = anime_rating_counts[anime_rating_counts >= min_anime_ratings].index

# Restreindre le dataset aux utilisateurs et animes filtrés
filtered_scores = user_score[user_score['user_id'].isin(filtered_users) & user_score['anime_id'].isin(filtered_animes)]

In [4]:
# Supprimer la variable
del user_score

# Forcer le garbage collector à libérer la mémoire
gc.collect()

# filtered_scores devient user_score on prend juste un échantillon des 1000 premiers animés
filtered_scores = filtered_scores.head(1000)

In [5]:
# Vérifier la taille du dataset après filtrage
print(f"Nombre d'utilisateurs filtrés : {len(filtered_users)}")
print(f"Nombre d'animes filtrés : {len(filtered_animes)}")
print(f"Nombre de notes après filtrage : {len(filtered_scores)}")

Nombre d'utilisateurs filtrés : 117568
Nombre d'animes filtrés : 598
Nombre de notes après filtrage : 1000


### Content Based Filtering

In [6]:
df_0 = filtered_scores.copy()

In [7]:
df_0.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
6,1,Xinil,53,Ai Yori Aoshi,7
7,1,Xinil,47,Akira,5
8,1,Xinil,591,Amaenaide yo!!,6


In [9]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 1905
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      1000 non-null   int64 
 1   Username     1000 non-null   object
 2   anime_id     1000 non-null   int64 
 3   Anime Title  1000 non-null   object
 4   rating       1000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


In [10]:
df_0.columns

Index(['user_id', 'Username', 'anime_id', 'Anime Title', 'rating'], dtype='object')

In [11]:
df_0.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
6,1,Xinil,53,Ai Yori Aoshi,7
7,1,Xinil,47,Akira,5
8,1,Xinil,591,Amaenaide yo!!,6


In [12]:
r = df_0.pivot_table(index='user_id', columns='anime_id', values='rating')
r.fillna(0, inplace=True)

In [15]:
model = NearestNeighbors(n_neighbors=10) 
model.fit(r)

NearestNeighbors(n_neighbors=10)

In [16]:
user_id = 1
user_vector = r.loc[user_id].values.reshape(1, -1)

In [17]:
distances, neighbors = model.kneighbors(user_vector, n_neighbors=10)

In [19]:
print(f"Les 10 utilisateurs les plus proches de l'utilisateur {user_id} sont :")
for i in range(1, len(distances[0])):
    similar_user_id = r.index[neighbors[0][i]]
    similarity = 100 / (1 + distances[0][i])
    print(f"Utilisateur {similar_user_id} avec une similarité de {similarity:.2f}%")

Les 10 utilisateurs les plus proches de l'utilisateur 1 sont :
Utilisateur 20 avec une similarité de 1.17%
Utilisateur 9 avec une similarité de 1.11%
Utilisateur 47 avec une similarité de 1.09%
Utilisateur 53 avec une similarité de 1.05%
Utilisateur 37 avec une similarité de 1.04%
Utilisateur 66 avec une similarité de 1.03%
Utilisateur 48 avec une similarité de 0.92%
Utilisateur 23 avec une similarité de 0.90%
Utilisateur 4 avec une similarité de 0.87%
